In [1]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp

Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 89.7MB/s]


In [2]:
!pip install kaggle


In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d yekenot/fasttext-crawl-300d-2m


mkdir: cannot create directory ‘/root/.kaggle’: File exists
fasttext-crawl-300d-2m.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!7z e fasttext-crawl-300d-2m.zip -o/content -r



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1545551987 bytes (1474 MiB)

Extracting archive: fasttext-crawl-300d-2m.zip
--
Path = fasttext-crawl-300d-2m.zip
Type = zip
Physical Size = 1545551987

  0%    
Would you like to replace the existing file:
  Path:     /content/crawl-300d-2M.vec
  Size:     4516698366 bytes (4308 MiB)
  Modified: 2019-09-27 20:43:22
with the file from archive:
  Path:     crawl-300d-2M.vec
  Size:     4516698366 bytes (4308 MiB)
  Modified: 2019-09-27 20:43:22
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? 

In [7]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# Reading glove vectors in python: https://stackoverflow.com/a/38230349/4084039
def fasttextModel(gloveFile):
    print ("Loading Fasttext Model")
    f = open(gloveFile,'r', encoding="utf8")
    model = {}#for storing word and the corresponding embedding vector for that word
    for line in f:
        splitLine = line.split()#splitting the line and storing it in a list
        word = splitLine[0]#getting the first element and storing it in word
        embedding = np.array([float(val) for val in splitLine[1:]])#obtaining corresponding vector for that word
        model[word] = embedding#storing word as key and embedding vector for that word as value
    print ("Done.",len(model)," words loaded!")
    return model
model = fasttextModel('/content/crawl-300d-2M.vec')

Loading Fasttext Model
Done. 2000000  words loaded!


In [9]:
df=pd.read_csv('preprocessed_data.csv')#creating DataFrame using preprocessed_data.csv

In [10]:
df.head(4)

,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [11]:
def preprocess(x):#removing last character
  x=x[:-1]
  return x

In [12]:
df['source']=df['source'].apply(preprocess)#preprocessing source data
df['target']=df['target'].apply(preprocess)#preprocessing target data

In [13]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [14]:
df.shape

(2000, 2)

In [15]:
def length(text):#for calculating the length of the sentence
    return len(str(text))

In [16]:
df=df[df['source'].apply(length)<170]#removing the datapoints where the source sentence length is greater than or equal to 170
df=df[df['target'].apply(length)<200]#removing the datapoints where the source sentence length is greater than or equal to 170

In [17]:
df.shape

(1990, 2)

In [18]:
df['target_in'] = '<start> ' + df['target'].astype(str)
df['target_out'] = df['target'].astype(str) + ' <end>'
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer
df.head()

,source,target,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?,<start> Do you want me to reserve seat for you...,Do you want me to reserve seat for you or not?...
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...,<start> Yeap. You reaching? We ordered some Du...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...,<start> They become more expensive already. Mi...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?,<start> I'm Thai. What do you do?,I'm Thai. What do you do? <end>
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...,<start> Hi! How did your week go? Haven't hear...,Hi! How did your week go? Haven't heard from y...


In [19]:
df=df.drop('target',axis=1)#removing the target column


In [20]:
df.head(4)

,source,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",<start> Do you want me to reserve seat for you...,Do you want me to reserve seat for you or not?...
1,Yup. U reaching. We order some durian pastry a...,<start> Yeap. You reaching? We ordered some Du...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,<start> They become more expensive already. Mi...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,<start> I'm Thai. What do you do?,I'm Thai. What do you do? <end>


In [21]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(df, test_size=0.01)#splitting the data in ratio 99:1

In [22]:
print(train.shape, validation.shape)
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both encoder output and decoder output
train.iloc[0]['target_in']= str(train.iloc[0]['target_in'])+' <end>'
train.iloc[0]['target_out']= str(train.iloc[0]['target_out'])+' <end>'

(1970, 3) (20, 3)


In [23]:
tknizer_source = Tokenizer()#creating tokenziation
tknizer_source.fit_on_texts(train['source'].values)#fitting on source data
tknizer_target = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')#creating tokenziation
tknizer_target.fit_on_texts(train['target_in'].values)#fitting on target data

In [24]:
vocab_size_target=len(tknizer_target.word_index.keys())#target vocab size
print(vocab_size_target)
vocab_size_source=len(tknizer_source.word_index.keys())#source vocab size
print(vocab_size_source)

3037
3703


In [25]:
tknizer_target.word_index['<start>'], tknizer_target.word_index['<end>']

(1, 1447)

In [26]:
encoder_embedding_matrix = np.zeros((vocab_size_source+1, 300))
for word, i in tknizer_source.word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        encoder_embedding_matrix[i] = embedding_vector

In [27]:
decoder_embedding_matrix = np.zeros((vocab_size_target+1, 300))
for word, i in tknizer_target.word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        decoder_embedding_matrix[i] = embedding_vector

In [28]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns encoder-outputs,encoder_final_state_h,encoder_final_state_c
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__()
        self.vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output=0
        self.lstm_state_h=0
        self.lstm_state_c=0
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True,name="embedding_layer_encoder",weights=[encoder_embedding_matrix],trainable=False)
        self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self,input_sequence,states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''
        
        input_embedd                           = self.embedding(input_sequence)
        lstm_state_h,lstm_state_c = states[0],states[1]
        self.lstm_output,lstm_state_h,lstm_state_c=self.lstm(input_embedd)
        return self.lstm_output,lstm_state_h,lstm_state_c

      

    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      return [np.zeros((batch_size,self.lstm_size)),np.zeros((batch_size,self.lstm_size))]


In [29]:
class Decoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,out_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Decoder LSTM layer
        super().__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_size = embedding_size
        self.lstm_size = lstm_size
        self.input_length = input_length
        # we are using embedding_matrix and not training the embedding layer
        self.embedding = tf.keras.layers.Embedding(input_dim=self.out_vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder",weights=[decoder_embedding_matrix],trainable=False)
        self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_sequences=True, return_state=True, name="Encoder_LSTM")
    
    def call(self,input_sequence,initial_states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to decoder_lstm
        
          returns -- decoder_output,decoder_final_state_h,decoder_final_state_c
        '''
        target_embedd = self.embedding(input_sequence)
        decoder_output,decoder_final_state_h,decoder_final_state_c = self.lstm(target_embedd, initial_state=[initial_states[0], initial_states[1]])
        return decoder_output,decoder_final_state_h,decoder_final_state_c

      


In [30]:
class Encoder_decoder(tf.keras.Model):
    
    def __init__(self,encoder_inputs_length,decoder_inputs_length, output_vocab_size,batch_size):
        
        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'
        super().__init__() # https://stackoverflow.com/a/27134600/4084039
        self.batch_size=batch_size
        self.encoder = Encoder(vocab_size_source+1,300,128,encoder_inputs_length)
        self.decoder = Decoder(vocab_size_target+1,300,128,decoder_inputs_length)
        self.dense   = tf.keras.layers.Dense(output_vocab_size, activation='softmax')
    
    
    def call(self,data):
        '''
        A. Pass the input sequence to Encoder layer -- Return encoder_output,encoder_final_state_h,encoder_final_state_c
        B. Pass the target sequence to Decoder layer with intial states as encoder_final_state_h,encoder_final_state_C
        C. Pass the decoder_outputs into Dense layer 
        
        Return decoder_outputs
        '''
        input,output = data[0], data[1]
        initial_state=self.encoder.initialize_states(self.batch_size)
        encoder_output, encoder_h, encoder_c = self.encoder(input,initial_state)
        decoder_output,decoder_final_state_h,decoder_final_state_c= self.decoder(output,[encoder_h, encoder_c])
        output                               = self.dense(decoder_output)
        return output
        

In [31]:
class Dataset:
    def __init__(self, df, tknizer_source, tknizer_target, source_len,target_len):
        self.encoder_inps = df['source'].values
        self.decoder_inps = df['target_in'].values
        self.decoder_outs = df['target_out'].values
        self.tknizer_target = tknizer_target
        self.tknizer_source = tknizer_source
        self.source_len = source_len
        self.target_len = target_len


    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_source.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_target.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_target.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.source_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.target_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.target_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [32]:
train_dataset = Dataset(train, tknizer_source, tknizer_target,39,43)
test_dataset  = Dataset(validation, tknizer_source, tknizer_target,39,43)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=20)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 39) (512, 43) (512, 43)


In [33]:
tf.config.experimental_run_functions_eagerly(True)


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [34]:
tf.config.run_functions_eagerly(True)


In [35]:
#Create an object of encoder_decoder Model class, 
# Compile the model and fit the model
model  = Encoder_decoder(encoder_inputs_length=39,decoder_inputs_length=43,output_vocab_size=vocab_size_target,batch_size=512)
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
train_steps=train.shape[0]//512
valid_steps=validation.shape[0]//20
model.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=40, validation_data=test_dataloader, validation_steps=valid_steps)
model.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/40
3/3 [==============================] - 1s 190ms/step - loss: 2.7104 - accuracy: 0.0480 - val_loss: 1.8951 - val_accuracy: 0.0794
Epoch 2/40
3/3 [==============================] - 1s 174ms/step - loss: 2.2126 - accuracy: 0.0686 - val_loss: 1.6150 - val_accuracy: 0.0833
Epoch 3/40
3/3 [==============================] - 1s 158ms/step - loss: 2.1212 - accuracy: 0.0788 - val_loss: 1.5956 - val_accuracy: 0.0913
Epoch 4/40
3/3 [==============================] - 1s 156ms/step - loss: 2.0691 - accuracy: 0.0857 - val_loss: 1.5894 - val_accuracy: 0.1071
Epoch 5/40
3/3 [==============================] - 1s 156ms/step - loss: 2.0377 - accuracy: 0.0935 - val_loss: 1.5779 - val_accuracy: 0.0992
Epoch 6/40
3/3 [==============================] - 1s 162ms/step - loss: 2.0195 - accuracy: 0.0941 - val_loss: 1.5682 - val_accuracy: 0.1071
Epoch 7/40
3/3 [==============================] - 1s 163ms/step - loss: 1.9993 - accuracy: 0.0988 - val_loss: 1.5554 - val_accuracy: 0.1071
Epoch 8/40
3/3 [====

In [36]:
batch_size=512
units=128

In [37]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to decoder
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predicted_out,state_h,state_c=model.layers[1](dec_input,states)
         pass the predicted_out to the dense layer
         update the states=[state_h,state_c]
         And get the index of the wordcc with maximum probability of the dense layer output, using the tokenizer(word index) get the word and then store it in a string.
         Update the input_to_decoder with current predictions
  F. Return the predicted sentence
  '''
  initial_state_enc=[np.zeros((batch_size,units)),np.zeros((batch_size,units))]
  inp_seq = tknizer_source.texts_to_sequences([input_sentence])
  inp_seq = pad_sequences(inp_seq,padding='post',maxlen=39)

  en_outputs,state_h , state_c = model.layers[0](tf.constant(inp_seq),initial_state_enc)
  cur_vec = tf.constant([[tknizer_target.word_index['<start>']]])
  pred = []
  #Here 43 is the max_length of the sequence
  for i in range(43):
    infe_output, state_h, state_c = model.layers[1](cur_vec,[state_h,state_c])
    infe_output = model.layers[2](infe_output)
    cur_vec = np.reshape(np.argmax(infe_output), (1, 1))
    pred.append(tknizer_target.index_word[cur_vec[0][0]])
    if(pred[-1]=='<end>'):
      break
    translated_sentence = ' '.join(pred)

  return translated_sentence

In [38]:
validation['target_in']

1266          <start> PJ. You're a Malay or Chinese, Rin?
1469                  <start> Help me look out for tubes.
769     <start> My sister and I are eating breakfast. ...
1800                 <start> Make that 3! For God's sake!
1064                <start> Are you all coming to school?
1622      <start> Hee. Ok. See you another time. Big hug.
1695    <start> Yes, I thought of it also but scare me...
1675    <start> Are you going to send a mail? Tomorrow...
1613                                <start> 26th of July.
1726                <start> Today mango got 10% discount.
821     <start> Don't ask. It is for a stupid reason. ...
1227    <start> Okay. But Tuesday I've got dinner. So ...
1286    <start> Oh. I see I see. I don't know. Message...
785     <start> Haha. I'm going to buy sandals. How to...
1107    <start> Ok, thanks. So do you think they would...
282     <start> XY and I are meeting for dinner. I'm i...
1151                                    <start> Yes. You?
1252    <start

In [39]:
for i in validation['source']:
  predicted=predict(i)
  print("The predicted output is: ",predicted)


The predicted output is:  i am still going to introduce
The predicted output is:  i think i will be late
The predicted output is:  i am going to go to the place to you
The predicted output is:  i will be late
The predicted output is:  i am going to chat
The predicted output is:  hi care to chat with you
The predicted output is:  hey i don't know i will be able to go out for the way
The predicted output is:  i don't know i will be able to go out
The predicted output is:  ok
The predicted output is:  no need to go
The predicted output is:  i don't know i will be able to go out for the way i don't know to go to go
The predicted output is:  hey i don't know i will be able to go out for the way i think i think i think i will go to go to go to go
The predicted output is:  hey i am going to go to watch
The predicted output is:  haha ok i don't know i will be able to go out for the place
The predicted output is:  hey i don't know i will be late
The predicted output is:  i don't know i will be 

In [40]:
# Predict on 1000 random sentences on test data and calculate the average BLEU score of these sentences.
import nltk.translate.bleu_score as bleu
bleu_scores_lst=[]
for i in validation[:]['source']:
  reference = [i.split(),] # the original
  predicted=predict(i)
  translation = predicted.split()
  values=bleu.sentence_bleu(reference, translation)
  bleu_scores_lst.append(values)

# https://www.nltk.org/_modules/nltk/translate/bleu_score.html

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [43]:
average_bleu_scores=sum(bleu_scores_lst)/len(bleu_scores_lst)
print("Average BLEU score of these 20 test data sentences is: ",average_bleu_scores)

Average BLEU score of these 20 test data sentences is:  0.17936797861081147


In [42]:
bleu_scores_lst


[0,
 0,
 0,
 0,
 0.668740304976422,
 0,
 0.5724063666159062,
 0.6051012508914458,
 0,
 0,
 0.4728708045015879,
 0.43092381945890607,
 0,
 0.23927141250362965,
 0,
 0,
 0,
 0.5980456132683322,
 0,
 0]

Character_Level:

In [11]:
df=pd.read_csv('preprocessed_data.csv')
df.head()

,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n
4,4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [12]:
def preprocess(x):
  x=x[:-1]
  return x

In [13]:
df['source']=df['source'].apply(preprocess)
df['target']=df['target'].apply(preprocess)

In [14]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [15]:
df.shape

(2000, 2)

In [16]:
def length(text):#for calculating the length of the sentence
    return len(str(text))

In [17]:
df=df[df['source'].apply(length)<170]
df=df[df['target'].apply(length)<200]

In [18]:
df.shape


(1990, 2)

In [19]:
df['target_in'] = '\t' + df['target'].astype(str)
df['target_out'] = df['target'].astype(str) + '\n'
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer
df.head()

,source,target,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?,\tDo you want me to reserve seat for you or not?,Do you want me to reserve seat for you or not?\n
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...,\tYeap. You reaching? We ordered some Durian p...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...,\tThey become more expensive already. Mine is ...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?,\tI'm Thai. What do you do?,I'm Thai. What do you do?\n
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...,\tHi! How did your week go? Haven't heard from...,Hi! How did your week go? Haven't heard from y...


In [20]:
df=df.drop('target',axis=1)


In [21]:
df.head(4)

,source,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",\tDo you want me to reserve seat for you or not?,Do you want me to reserve seat for you or not?\n
1,Yup. U reaching. We order some durian pastry a...,\tYeap. You reaching? We ordered some Durian p...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,\tThey become more expensive already. Mine is ...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,\tI'm Thai. What do you do?,I'm Thai. What do you do?\n


In [22]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(df, test_size=0.01)

In [23]:
print(train.shape, validation.shape)
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both encoder output and decoder output
train.iloc[0]['target_in']= str(train.iloc[0]['target_in'])+'\n'
train.iloc[0]['target_out']= str(train.iloc[0]['target_out'])+'\n'

(1970, 3) (20, 3)


In [24]:
tknizer_source = Tokenizer(filters=None,char_level=True,lower=False)
tknizer_source.fit_on_texts(train['source'].values)
tknizer_target = Tokenizer(filters=None,char_level=True,lower=False)
tknizer_target.fit_on_texts(train['target_in'].values)

In [25]:
vocab_size_target=len(tknizer_target.word_index.keys())
print(vocab_size_target)
vocab_size_source=len(tknizer_source.word_index.keys())
print(vocab_size_source)

91
101


In [26]:
tknizer_target.word_index['\t'], tknizer_target.word_index['\n']

(20, 85)

In [48]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns encoder-outputs,encoder_final_state_h,encoder_final_state_c
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__()
        self.vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output=0
        self.lstm_state_h=0
        self.lstm_state_c=0
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder",trainable=True)
        self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self,input_sequence,states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''

        input_embedd                           = self.embedding(input_sequence)
        lstm_state_h,lstm_state_c = states[0],states[1]
        self.lstm_output,lstm_state_h,lstm_state_c=self.lstm(input_embedd)
        return self.lstm_output,lstm_state_h,lstm_state_c

      

    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      return [np.zeros((batch_size,self.lstm_size)),np.zeros((batch_size,self.lstm_size))]


In [50]:
class Decoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,out_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Decoder LSTM layer
        super().__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_size = embedding_size
        self.lstm_size = lstm_size
        self.input_length = input_length
        # we are using embedding_matrix and not training the embedding layer
        self.embedding = tf.keras.layers.Embedding(input_dim=self.out_vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", trainable=True)
        self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_sequences=True, return_state=True, name="Encoder_LSTM")
    
    def call(self,input_sequence,initial_states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to decoder_lstm
        
          returns -- decoder_output,decoder_final_state_h,decoder_final_state_c
        '''
        target_embedd = self.embedding(input_sequence)
        decoder_output,decoder_final_state_h,decoder_final_state_c = self.lstm(target_embedd, initial_state=[initial_states[0], initial_states[1]])
        return decoder_output,decoder_final_state_h,decoder_final_state_c

      


In [53]:
class Encoder_decoder(tf.keras.Model):
    
    def __init__(self,encoder_inputs_length,decoder_inputs_length, output_vocab_size,batch_size):
        
        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'
        super().__init__() # https://stackoverflow.com/a/27134600/4084039
        self.batch_size=batch_size
        self.encoder = Encoder(vocab_size_source+1,300,100,encoder_inputs_length)
        self.decoder = Decoder(vocab_size_target+1,300,100,decoder_inputs_length)
        self.dense   = tf.keras.layers.Dense(output_vocab_size, activation='softmax')
    
    
    def call(self,data):
        '''
        A. Pass the input sequence to Encoder layer -- Return encoder_output,encoder_final_state_h,encoder_final_state_c
        B. Pass the target sequence to Decoder layer with intial states as encoder_final_state_h,encoder_final_state_C
        C. Pass the decoder_outputs into Dense layer 
        
        Return decoder_outputs
        '''
        input,output = data[0], data[1]
        initial_state=self.encoder.initialize_states(self.batch_size)
        encoder_output, encoder_h, encoder_c = self.encoder(input,initial_state)
        decoder_output,decoder_final_state_h,decoder_final_state_c= self.decoder(output,[encoder_h, encoder_c])
        output                               = self.dense(decoder_output)
        return output
        

In [54]:
class Dataset:
    def __init__(self, df, tknizer_source, tknizer_target, source_len,target_len):
        self.encoder_inps = df['source'].values
        self.decoder_inps = df['target_in'].values
        self.decoder_outs = df['target_out'].values
        self.tknizer_target = tknizer_target
        self.tknizer_source = tknizer_source
        self.source_len = source_len
        self.target_len = target_len


    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_source.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_target.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_target.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.source_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.target_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.target_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [57]:
train_dataset = Dataset(train, tknizer_source, tknizer_target,170,200)
test_dataset  = Dataset(validation, tknizer_source, tknizer_target,170,200)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=20)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 170) (512, 200) (512, 200)


In [58]:
#Create an object of encoder_decoder Model class, 
# Compile the model and fit the model
model  = Encoder_decoder(encoder_inputs_length=170,decoder_inputs_length=200,output_vocab_size=vocab_size_target,batch_size=512)
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
train_steps=train.shape[0]//512
valid_steps=validation.shape[0]//20
model.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=100, validation_data=test_dataloader, validation_steps=valid_steps)
model.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/100
3/3 [==============================] - 1s 348ms/step - loss: 1.4916 - accuracy: 0.1226 - val_loss: 1.0614 - val_accuracy: 0.1744
Epoch 2/100
3/3 [==============================] - 1s 341ms/step - loss: 1.1787 - accuracy: 0.1884 - val_loss: 1.0143 - val_accuracy: 0.1744
Epoch 3/100
3/3 [==============================] - 1s 342ms/step - loss: 1.1388 - accuracy: 0.1823 - val_loss: 0.9851 - val_accuracy: 0.1776
Epoch 4/100
3/3 [==============================] - 1s 332ms/step - loss: 1.1065 - accuracy: 0.1901 - val_loss: 0.9493 - val_accuracy: 0.2036
Epoch 5/100
3/3 [==============================] - 1s 338ms/step - loss: 1.0596 - accuracy: 0.2329 - val_loss: 0.9110 - val_accuracy: 0.2466
Epoch 6/100
3/3 [==============================] - 1s 330ms/step - loss: 1.0142 - accuracy: 0.2621 - val_loss: 0.8720 - val_accuracy: 0.2620
Epoch 7/100
3/3 [==============================] - 1s 330ms/step - loss: 0.9679 - accuracy: 0.2843 - val_loss: 0.8407 - val_accuracy: 0.2960
Epoch 8/100
3

In [59]:
batch_size=512
units=100

In [60]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to decoder
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predicted_out,state_h,state_c=model.layers[1](dec_input,states)
         pass the predicted_out to the dense layer
         update the states=[state_h,state_c]
         And get the index of the wordcc with maximum probability of the dense layer output, using the tokenizer(word index) get the word and then store it in a string.
         Update the input_to_decoder with current predictions
  F. Return the predicted sentence
  '''
  initial_state_enc=[np.zeros((batch_size,units)),np.zeros((batch_size,units))]
  inp_seq = tknizer_source.texts_to_sequences([input_sentence])
  inp_seq = pad_sequences(inp_seq,padding='post',maxlen=170)

  en_outputs,state_h , state_c = model.layers[0](tf.constant(inp_seq),initial_state_enc)
  cur_vec = tf.constant([[tknizer_target.word_index['\t']]])
  pred = []
  #Here 200 is the max_length of the sequence
  for i in range(200):
    infe_output, state_h, state_c = model.layers[1](cur_vec,[state_h,state_c])
    infe_output = model.layers[2](infe_output)
    cur_vec = np.reshape(np.argmax(infe_output), (1, 1))
    pred.append(tknizer_target.index_word[cur_vec[0][0]])
    if(pred[-1]=='\n'):
      break
    translated_sentence = ''.join(pred)

  return translated_sentence

In [69]:
for i in validation['source']:
  print("The Actual Output is:")
  print(i)
  print("The Predicted Output is:")
  pred=predict(i)
  print(pred)
  print('>'*100)


The Actual Output is:
Y sad sad
The Predicted Output is:
Yes. I am not to meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to meet yo
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Actual Output is:
Ding me on ya break fassyole! Blacko from londn
The Predicted Output is:
Anyone meet you all the meet you all the meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to meet you are to
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Actual Output is:
OH YEAH,AND HAV A GREAT TIME IN NEWQUAY-SEND ME A POSTCARD !1 LOOK AFTER ALL THE GIRLS WHILE IM GONE(U KNOW THE 1IM TALKIN BOUT!)
The Predicted Output is:
Hell you go to go to meet you all the meet you all the meet you all

In [70]:
# Predict on 1000 random sentences on test data and calculate the average BLEU score of these sentences.
import nltk.translate.bleu_score as bleu
bleu_scores_lst=[]
for i in validation[:]['source']:
  reference = [i.split(),] # the original
  predicted=predict(i)
  translation = predicted.split()
  values=bleu.sentence_bleu(reference, translation)
  bleu_scores_lst.append(values)

# https://www.nltk.org/_modules/nltk/translate/bleu_score.html

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [71]:
average_bleu_scores=sum(bleu_scores_lst)/len(bleu_scores_lst)
print("Average BLEU score of these 1000 test data sentences is: ",average_bleu_scores)

Average BLEU score of these 1000 test data sentences is:  0.1318701360495755


In [72]:
bleu_scores_lst

[0,
 0,
 0,
 0,
 0.3742031646082125,
 0,
 0.3760603093086394,
 0.1690308509457033,
 0,
 0.3742031646082125,
 0,
 0.4494780405208269,
 0,
 0,
 0,
 0,
 0.447213595499958,
 0,
 0,
 0.447213595499958]